In [1]:
from langchain_community.llms import Ollama
from langchain_core.output_parsers import JsonOutputParser
from langchain.output_parsers import YamlOutputParser

In [30]:
llm = Ollama(model="gemma:7b")

In [31]:
from langchain_core.prompts import ChatPromptTemplate
import re

def extract_cypher_scripts(text):
    cypher_list = list()
    for line in text.split("\n"):
        if 'CREATE' in line or 'MATCH' in line or 'CONNECT' in line:
            cypher_list.append(line)
            
    return cypher_list

## CoT Knowledge Grapg Creation

In [32]:
import pandas as pd
import numpy as np
cypher_list = list()
contexts = pd.read_csv('../context_group.csv').groupby('group_id')
context_outputs = list()
for name, group_context in contexts:
    context = "\n".join(group['context'].to_list())
    prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a ontologist. You build knowledge graph from plain text. \n 
     Task 1: Extract Entities from plain text as possible as atomic. \n
     Task 2: Define Cypher Node Type to each Entity.\n
     Task 3: Add features of each Entity from plain text. \n
     Task 4: Extract relationship between Entities from plain text as possible as atomic. \n\n
     Task 5: Convert all outputs to Neo4j Cypher Creation Format"""),
    ("system", "Let's think step by step."),
    ("user", "{input}")
    ])
    chain = prompt | llm 
    output = chain.invoke({"input": context})
    print(output)
    print('--------------------------------')
    cypher_scripts = extract_cypher_scripts(output.replace('`','"'))
    context_outputs.append(cypher_scripts)
    

## Knowledge Graph Extraction from Text: Beyoncé Giselle Knowles-Carter

**Task 1: Entity Extraction:**

* **Entities:**
    * Beyoncé Giselle Knowles-Carter
    * American singer
    * Songwriter
    * Record producer
    * Actress
    * Houston, Texas
    * Destiny's Child
    * Mathew Knowles
    * Grammy Awards
    * Billboard Hot 100
    * Dangerously in Love
    * Déjà Vu
    * B'Day
    * Dreamgirls
    * The Pink Panther
    * Obsessed
    * I Am... Sasha Fierce
    * 4
    * Beyoncé

**Task 2: Cypher Node Type:**

* **Beyoncé:** Person
* **American singer:** Person
* **Songwriter:** Person
* **Record producer:** Person
* **Actress:** Person
* **Houston, Texas:** Location
* **Destiny's Child:** Group
* **Mathew Knowles:** Manager
* **Grammy Awards:** Award
* **Billboard Hot 100:** Chart
* **Dangerously in Love:** Album
* **Déjà Vu:** Song
* **B'Day:** Album
* **Dreamgirls:** Movie
* **The Pink Panther:** Movie
* **Obsessed:** Movie
* **I Am... Sasha Fierce:** Album
* **4:** Alb

In [33]:
import pickle 

def save_list_to_file(my_list, file_path):
    with open(file_path, 'wb') as file:
        pickle.dump(my_list, file)
    print(f'List saved to {file_path}')
    
def read_list_from_file(file_path):
    with open(file_path, 'rb') as file:
        my_list = pickle.load(file)
    print(f'List read from {file_path}')
    return my_list


In [34]:

save_path = './Automated_KG_list_Group.pkl'
save_list_to_file(context_outputs, save_path)
context_read


List saved to ./Automated_KG_list_Group.pkl


NameError: name 'context_read' is not defined

In [13]:
context_read[0]

["CREATE (b:Person {name: 'Beyoncé Giselle Knowles-Carter', born: 1981})",
 "CREATE (h:Location {name: 'Houston, Texas'})",
 "CREATE (g:Group {name: 'Destiny''s Child'})",
 "CREATE (m:Manager {name: 'Mathew Knowles'})",
 "CREATE (a:Album {name: 'Dangerously in Love'})",
 'CREATE (b)-[:BORN_AND_RAISED_IN]->(h)',
 'CREATE (b)-[:PERFORMED_AS]->(g)',
 'CREATE (b)-[:MANAGED_BY]->(m)',
 'CREATE (g)-[:BECAME]->(a)',
 'CREATE (a)-[:ESTABLISHED]->(b)']